# 18650 Battery Models

## Aryan, Avery, Cormac, & Tyler

In [4]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import pandas
import numpy
import sklearn

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\tyler\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.